In [1]:
import time
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from scipy.signal import savgol_filter

%matplotlib inline

from astropy.time import Time, TimeDelta
# import astropy.units as u

from lsst.summit.utils.tmaUtils import TMAEventMaker, getSlewsFromEventList
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.sitcom.vandv.mount import get_slew_from_mtmount, get_slew_pairs

from lsst.sitcom import vandv
from lsst_efd_client import EfdClient

In [2]:
# imports that might be useful later but not currently
# import sys, os, asyncio, glob
# from datetime import datetime
# import subprocess
# import pickle as pkl

In [3]:
def get_univariate_splines(times, positions, velocities, interpPoints, kernel, smoothingFactor):
    try:
        posSpline = UnivariateSpline(times, position, s=0)
    except:
        #if there are duplicate time measurements remove them  (this occured on 
        # 23/11/22-23 and 21-22)
        times, indexes=np.unique(times, return_index=True)
        positions=positions[indexes]
        velocities=velocities[indexes]
        
        posSpline = UnivariateSpline(times, positions, s=0)
        velSpline1  = UnivariateSpline(times, velocities, s=0) 
            
    # Now smooth the derivative before differentiating again
    smoothedVel = np.convolve(velSpline1(interpPoints), kernel, mode='same')
    velSpline = UnivariateSpline(interpPoints, smoothedVel, s=smoothingFactor)
    accSpline1 = velSpline.derivative(n=1)
    smoothedAcc = np.convolve(accSpline1(interpPoints), kernel, mode='same')
    # Now smooth the derivative before differentiating again
    accSpline = UnivariateSpline(interpPoints, smoothedAcc, s=smoothingFactor)
    jerkSpline = accSpline.derivative(n=1)
    return posSpline(interpPoints), velSpline(interpPoints), accSpline(interpPoints), jerkSpline(interpPoints)

In [4]:
# define the savgol filter first
def savgolFilter(times, positions,interpPoints, window=200, deriv=1, smoothingFactor = 0.01): 
    positionSpline = UnivariateSpline(times, positions, s=smoothingFactor)(interpPoints) 
    derivativePoints = savgol_filter(positionSpline, window_length=window, mode="mirror",
                                    deriv=deriv, polyorder=3, delta=(interpPoints[1]-interpPoints[0])) 
    return derivativePoints#interp1d(times,derivativePoints)(interpPoints)

def get_savgol_splines(times, positions, interpPoints):
    posSpline = UnivariateSpline(times, positions, s=0)(interpPoints)
    velSpline = savgolFilter(times, positions, interpPoints, window=200, deriv=1, smoothingFactor=0.01)
    accSpline = savgolFilter(times, positions, interpPoints, window=200, deriv=2, smoothingFactor=0.01)
    jerkSpline = savgolFilter(times, positions, interpPoints, window=200, deriv=3, smoothingFactor=0.01)
    return posSpline, velSpline, accSpline, jerkSpline

In [5]:
def get_spline_frame(dayObs, index, spline_method, fullAzTimestamp, fullElTimestamp, az_position, az_velocity, el_position, el_velocity):
    npoints = int(np.max([np.round((fullAzTimestamp[-1]-fullAzTimestamp[0])/0.01/1e3,0)*1e3, 4000])) # clarify what this is doing
    plotAzXs = np.linspace(fullAzTimestamp[0], fullAzTimestamp[-1], npoints)
    plotElXs = np.linspace(fullElTimestamp[0], fullElTimestamp[-1], npoints)
    
    kernel_size = len(fullAzTimestamp)
    kernel = np.ones(kernel_size)/kernel_size
    s = 0 # smoothing factor
    
    if spline_method == "spline":
        # input: times, positions, velocities, interpPoints, kernel, smoothing factor
        azPosSpline, azVelSpline, azAccSpline, azJerkSpline = get_univariate_splines(fullAzTimestamp, az_position, az_velocity, plotAzXs, kernel, s) 
        elPosSpline, elVelSpline, elAccSpline, elJerkSpline = get_univariate_splines(fullElTimestamp, el_position, el_velocity, plotElXs, kernel, s)
    elif spline_method == "savgol":
        azPosSpline, azVelSpline, azAccSpline, azJerkSpline = get_savgol_splines(fullAzTimestamp, az_position, plotAzXs)
        elPosSpline, elVelSpline, elAccSpline, elJerkSpline = get_savgol_splines(fullElTimestamp, el_position, plotElXs)
    else:
        assert False, spline_method + " is not a valid spline method. Use either \"spline\" or \"savgol\"."
    
    # try replacing az_timestamp.values[0] with fullAzTimestamp[0]
    spline_frame=pd.DataFrame({
                "slew_index":index,
                "day":dayObs, # TO-DO: confirm this is correct usage. Is another format necessary?
                "azZeroTime":fullAzTimestamp.values[0],
                "elZeroTime":fullElTimestamp.values[0],
                "azTime":plotAzXs,
                "azPosition":azPosSpline,
                "azVelocity":azVelSpline,          
                "azAcceleration":azAccSpline,
                "azJerk":azJerkSpline,
                "elTime":plotElXs,
                "elPosition":elPosSpline,
                "elVelocity":elVelSpline,          
                "elAcceleration":elAccSpline,
                "elJerk":elJerkSpline}
    )
    
    return spline_frame

In [6]:
# define start and end date for the query
# TO-DO: actually implement this in the query
t_start = Time("2023-06-02T03:20:00")
t_end =  Time("2023-06-02T10:00:00")

# generate the client that will interact with the EFD
client = EfdClient('usdf_efd')

Identify the slews using Merlin's summit_utils

In [7]:
# TO-DO: add ways to offer a range of time to look using these tools

# create a list for multiple days
days = [
    20230621
    # 20230622,
    # 20230623,
    # 20230627,
    # 20230628,
    # 20230629]
]

# why does 20230620 raise block ID errors?

eventMaker = TMAEventMaker()
# replace client with eventMaker.client()

topic_az = "lsst.sal.MTMount.azimuth"
topic_el = "lsst.sal.MTMount.elevation"

spline_frame = pd.DataFrame()

# query for all days
for day in days:
    dayEvents = eventMaker.getEvents(day)
    print(f'Found {len(dayEvents)} events for {day=}')
    
    # filter only slew events from the event list
    slew_events = getSlewsFromEventList(dayEvents)
    print(f'Found {len(slew_events)} slews for {day=}')
    
    for i in range(len(slew_events)):
        data_az = getEfdData(client, topic_az, event=slew_events[i])
        data_el = getEfdData(client, topic_el, event=slew_events[i])
        
        # only pull data from slew event if the event has data
        if data_az.shape[0] != 0:
        
            az_position = data_az["actualPosition"]
            az_velocity = data_az["actualVelocity"]
            az_timestamp = data_az["timestamp"]
            # az_timestamp_relative = az_timestamp - az_timestamp[0]

            el_position = data_el["actualPosition"]
            el_velocity = data_el["actualVelocity"]
            el_timestamp = data_el["timestamp"]
            # el_timestamp_relative = el_timestamp - el_timestamp[0]
            print('done: length of data is ' + str(len(az_position)))

            spline_frame_single = get_spline_frame(day, i, "spline", az_timestamp, el_timestamp, az_position, az_velocity, el_position, el_velocity)
            spline_frame = pd.concat([spline_frame, spline_frame_single], ignore_index=False) #True

Found 53 events for day=20230621
Found 47 slews for day=20230621
done: length of data is 178
done: length of data is 507
done: length of data is 784
done: length of data is 25875
done: length of data is 547
done: length of data is 2077
done: length of data is 559
done: length of data is 1685
done: length of data is 172
done: length of data is 440
done: length of data is 14616
done: length of data is 14390
done: length of data is 29611
done: length of data is 29527
done: length of data is 29613
done: length of data is 27959
done: length of data is 1674


done: length of data is 335
done: length of data is 698
done: length of data is 601
done: length of data is 457
done: length of data is 2792


done: length of data is 420
done: length of data is 691
done: length of data is 682
done: length of data is 3810
done: length of data is 29433
done: length of data is 29466
done: length of data is 29628
done: length of data is 29611
done: length of data is 2003


done: length of data is 485
done: length of data is 689
done: length of data is 770
done: length of data is 2545
done: length of data is 172
done: length of data is 444
done: length of data is 472
done: length of data is 168
done: length of data is 524
done: length of data is 29438


In [8]:
print("Shape of final frame: " + str(spline_frame.shape))
# print(spline_frame)
spline_test = spline_frame.loc[((spline_frame['day']==20230621) & (spline_frame['slew_index']==2))]
print(spline_test)

Shape of final frame: (2129000, 14)
      slew_index       day    azZeroTime    elZeroTime        azTime   
0              2  20230621  1.687389e+09  1.687389e+09  1.687389e+09  \
1              2  20230621  1.687389e+09  1.687389e+09  1.687389e+09   
2              2  20230621  1.687389e+09  1.687389e+09  1.687389e+09   
3              2  20230621  1.687389e+09  1.687389e+09  1.687389e+09   
4              2  20230621  1.687389e+09  1.687389e+09  1.687389e+09   
...          ...       ...           ...           ...           ...   
4995           2  20230621  1.687389e+09  1.687389e+09  1.687389e+09   
4996           2  20230621  1.687389e+09  1.687389e+09  1.687389e+09   
4997           2  20230621  1.687389e+09  1.687389e+09  1.687389e+09   
4998           2  20230621  1.687389e+09  1.687389e+09  1.687389e+09   
4999           2  20230621  1.687389e+09  1.687389e+09  1.687389e+09   

      azPosition  azVelocity  azAcceleration        azJerk        elTime   
0     151.913587    0.0

Identify maxmimum kinematic values and compare to values expected in current system design

In [9]:
slew_num = []
day_num = []
slew_time = [] # final time

az_vel_max = []
el_vel_max = []

az_acc_max = []
el_acc_max = []

az_jerk_max = []
el_jerk_max = []

for i in np.unique(spline_frame['day']):
    slew_day = (spline_frame['day']==i) # list of booleans for when day is equal to i
    for j in np.unique(spline_frame['slew_index'][slew_day]):
        slew_id = slew_day & (spline_frame['slew_index']==j)
        
        az_vel_max.append(abs(spline_frame.loc[slew_id,"azVelocity"]).max())
        el_vel_max.append(abs(spline_frame.loc[slew_id,"elVelocity"]).max())
        
        az_acc_max.append(abs(spline_frame.loc[slew_id,"azAcceleration"]).max())
        el_acc_max.append(abs(spline_frame.loc[slew_id,"elAcceleration"]).max())

        az_jerk_max.append(abs(spline_frame.loc[slew_id,"azJerk"]).max())
        el_jerk_max.append(abs(spline_frame.loc[slew_id,"elJerk"]).max())
        
        slew_num.append(j)
        day_num.append(i)
        
        slew_time.append(np.max([spline_frame.loc[slew_id,"azTime"].max(),spline_frame.loc[slew_id,"elTime"].max()]))

max_frame=pd.DataFrame({
    "day":day_num,
    "slew":slew_num,
    "az_vel":az_vel_max,
    "az_acc":az_acc_max,
    "az_jerk":az_jerk_max,
    "el_vel":el_vel_max,
    "el_acc":el_acc_max,
    "el_jerk":el_jerk_max})

Graphs Galore

In [10]:
# generate histograms for max slews
def plot_max_hist(spline_frame, limitsBool):

    fig,axs = plt.subplots(3, 2, dpi=175, figsize=(10,5), sharex=False)
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    # TO-DO: add start time and date to title
    plt.suptitle("Maximums", fontsize = 14)

    # make it easier to change variables across subplots
    velbins=np.linspace(0, 1, 100)
    accbins=np.linspace(0, 1, 100)
    jerkbins=np.linspace(0, 2, 100)
    
    plt.subplot(3,2,1)
    plt.hist(max_frame["az_vel"], color="tab:blue", bins=velbins)
    plt.title(f"Azimuth")
    plt.ylabel("Velcoity Count")

    plt.subplot(3,2,2)
    plt.hist(max_frame["el_vel"], color="tab:blue", bins=velbins)
    plt.title(f"Elevation")

    plt.subplot(3,2,3)
    plt.hist(max_frame["az_acc"], color="tab:blue", bins=accbins)
    if limitsBool == True:
        plotAzDesignlim("design_velocity", "max_velocity", azRelativeTimes[0], azRelativeTimes.iloc[[-1]], design_color, max_color)
    plt.ylabel("Acceleration Count")

    plt.subplot(3,2,4)
    plt.hist(max_frame["el_acc"], color="tab:blue", bins=accbins)
    if limitsBool == True:
        plotElDesignlim("design_velocity", "max_velocity", elRelativeTimes[0], elRelativeTimes.iloc[[-1]], design_color, max_color)

    plt.subplot(3,2,5)
    plt.hist(max_frame["az_jerk"], color="tab:blue", bins=jerkbins)
    if limitsBool == True:
        plotAzDesignlim("design_acceleration", "max_acceleration", azRelativeTimes[0], azRelativeTimes.iloc[[-1]], design_color, max_color)
    plt.ylabel("Jerk Count")

    plt.subplot(3,2,6)
    plt.hist(max_frame["az_jerk"], color="tab:blue", bins=jerkbins)
    if limitsBool == True:
        plotElDesignlim("design_acceleration", "max_acceleration", elRelativeTimes[0], elRelativeTimes.iloc[[-1]], design_color, max_color)

    plt.show()

In [11]:
plot_max_hist(slew_spline, False)

NameError: name 'slew_spline' is not defined

In [ ]:
# define limits from science requirements document for plotting
# units in deg/s - deg/s^2 - deg/s^3
el_limit_dict={
    "max_velocity": 5.25,
    "max_acceleration": 5.25,
    "max_jerk": 21,
    "design_velocity": 3.5,
    "design_acceleration": 3.5,
    "design_jerk": 14,
}
az_limit_dict={
    "max_velocity": 10.5,
    "max_acceleration": 10.5,
    "max_jerk": 42,
    "design_velocity": 7,
    "design_acceleration": 7,
    "design_jerk": 28,
}

In [ ]:
# define functions to add limits to slew profile for neatness
def plotAzDesignlim(design_input, max_input, xmin, xmax, design_color, max_color):
    plt.hlines(az_limit_dict[design_input], xmin=xmin, xmax=xmax, color = design_color)
    plt.hlines(-az_limit_dict[design_input], xmin=xmin, xmax=xmax, color = design_color)
    plt.hlines(az_limit_dict[max_input], xmin=xmin, xmax=xmax, color = max_color)
    plt.hlines(-az_limit_dict[max_input], xmin=xmin, xmax=xmax, color = max_color)
    
def plotElDesignlim(design_input, max_input, xmin, xmax, design_color, max_color):
    plt.hlines(el_limit_dict[design_input], xmin=xmin, xmax=xmax, color = design_color)
    plt.hlines(-el_limit_dict[design_input], xmin=xmin, xmax=xmax, color = design_color)
    plt.hlines(el_limit_dict[max_input], xmin=xmin, xmax=xmax, color = max_color)
    plt.hlines(-el_limit_dict[max_input], xmin=xmin, xmax=xmax, color = max_color)

In [ ]:
# pick a slew to show a single slew motion analysis
# loc[((df['col1'] == 'A') & (df['col2'] == 'G'))]
slew_spline = spline_frame.loc[((spline_frame['day']==20230621) & (spline_frame['slew_index']==2))]
print(slew_spline)

In [ ]:
# Generate plots for a slew profile in both azimuth and elevation
# Position, velocity, acceleration, and jerk
# limitsBool when set to true adds spec limits to graph
# TO-DO: add bool to offer choice between showing plot or saving plot to a file

def slew_profile_plot(spline_frame, limitsBool):
    # get the relative times for the x-axis
    azRelativeTimes = spline_frame['azTime'] - spline_frame['azZeroTime']
    elRelativeTimes = spline_frame['elTime'] - spline_frame['elZeroTime']

    fig,axs = plt.subplots(4, 2, dpi=175, figsize=(10,5), sharex=True)
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    # TO-DO: add start time and date to title
    plt.suptitle("MT Mount Slew Example", fontsize = 14)

    # make it easier to change variables across subplots
    mark = "x"
    mark_color = "purple"
    mark_size = 30
    line_width = 2
    az_color = "red"
    el_color = "blue"
    design_color = "green"
    max_color = "orange"
    opacity = 0.5
    
    plt.subplot(4,2,1)
    plt.plot(azRelativeTimes, spline_frame['azPosition'], lw=line_width, color=az_color, label='Spline fit')
    plt.scatter(azRelativeTimes, spline_frame['azPosition'], marker=mark, color=mark_color,alpha=opacity, s=mark_size, label='Measured points')
    plt.title(f"Azimuth")
    plt.ylabel("Degrees")

    plt.subplot(4,2,2)
    plt.plot(elRelativeTimes, spline_frame['elPosition'], lw=line_width, color=el_color, label='Spline fit')
    plt.scatter(elRelativeTimes, spline_frame['elPosition'], marker=mark, color=mark_color,alpha=opacity, s=mark_size, label='Measured points')
    plt.title(f"Elevation")

    plt.subplot(4,2,3)
    plt.plot(azRelativeTimes, spline_frame['azVelocity'], lw=line_width, color=az_color, label='Spline fit')
    plt.scatter(azRelativeTimes, spline_frame['azVelocity'], marker=mark, color=mark_color,alpha=opacity, s=mark_size, label='Measured points')
    if limitsBool == True:
        plotAzDesignlim("design_velocity", "max_velocity", azRelativeTimes[0], azRelativeTimes.iloc[[-1]], design_color, max_color)
    plt.ylabel("Degrees/sec")

    plt.subplot(4,2,4)
    plt.plot(elRelativeTimes, spline_frame['elVelocity'], lw=line_width, color=el_color, label='Spline fit')
    plt.scatter(elRelativeTimes, spline_frame['elVelocity'], marker=mark, color=mark_color,alpha=opacity, s=mark_size, label='Measured points')
    if limitsBool == True:
        plotElDesignlim("design_velocity", "max_velocity", elRelativeTimes[0], elRelativeTimes.iloc[[-1]], design_color, max_color)

    plt.subplot(4,2,5)
    plt.plot(azRelativeTimes, spline_frame['azAcceleration'], lw=line_width, color=az_color, label='Spline fit')
    if limitsBool == True:
        plotAzDesignlim("design_acceleration", "max_acceleration", azRelativeTimes[0], azRelativeTimes.iloc[[-1]], design_color, max_color)
    plt.ylabel("Degrees/sec^2")

    plt.subplot(4,2,6)
    plt.plot(elRelativeTimes, spline_frame['elAcceleration'], lw=line_width, color=el_color, label='Spline fit')
    if limitsBool == True:
        plotElDesignlim("design_acceleration", "max_acceleration", elRelativeTimes[0], elRelativeTimes.iloc[[-1]], design_color, max_color)

    plt.subplot(4,2,7)
    plt.plot(azRelativeTimes, spline_frame['azJerk'], lw=line_width, color=az_color, label='Spline fit')
    if limitsBool == True:
        plotAzDesignlim("design_jerk", "max_jerk", azRelativeTimes[0], azRelativeTimes.iloc[[-1]], design_color, max_color)
    plt.ylabel("Degrees/sec^3")

    plt.subplot(4,2,8)
    plt.plot(elRelativeTimes, spline_frame['elJerk'], lw=line_width, color=el_color, label='Spline fit')
    if limitsBool == True:
        plotElDesignlim("design_jerk", "max_jerk", elRelativeTimes[0], elRelativeTimes.iloc[[-1]], design_color, max_color)

    plt.show()

In [ ]:
slew_profile_plot(slew_spline, True)

In [ ]:
fig.clear()

In [ ]:
# create alerts for slews that don't behave. Definition of not behaving tbd. 